# Hangman NLP Algorithm

## Description:
To tackle the hangman word guessing problem, I used the n-gram NLP probabilistic model. In this algorithm, a n-gram will be defined as a n-letter substring of a certain word instead of n separate words. To initialize and train the model, the algorithm first iterates through all the possible 1-gram to 9-gram substrings of all the words in the training dictionary and stores the substring frequencies in a map. The range is 1-gram to 9-gram substrings because most English words fall in the length of 1 to 9 letters and from further testing, I found out that any 10-gram or more will have negative effects on the accuracy of the algorithm. 

After storing the frequencies, the algorithm guesses the next letter for a given word by iterating through all the 1-gram to 9-gram substrings of the word with one missing letter.

For example, the possible considerations of a 3-gram substring would be:
[Letter][Letter][ _ ] or
[Letter][ _ ][Letter] or
[ _ ][Letter][Letter]

Then, it tries to fit any possible non-guessed letter into that missing letter of the n-gram of the word by calculating the probability that the letter is in that position. This is accomplished by our frequency map that we initialized in the beginning. We store the probabilities of each letter occurring in all n-grams of the word and we take the letter with the maximum probability to be our guess.



In [ ]:
import time
import requests
import json
import re

class HangmanAPI(object):
    def __init__(self):
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        
        self.current_dictionary = []

        # specifics for NLP n-gram probabilistic model
        self.prob = {}
        self.ngram = {}
        self.ngram_lim = 9
        self.weights = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

        self.initialize_ngrams(self.full_dictionary)            

    def guess(self, word): # word input example: "_ p p _ e "

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # convert to lowercase
        clean_word = clean_word.lower()

        self.prob.clear()

        for c in range(ord('a'),ord('z')+1):
            if(chr(c) not in self.guessed_letters):
                self.prob[chr(c)] = 0

        guess_letter = self.compute_ngrams(clean_word)

        return guess_letter

        
    def initialize_ngrams(self, dict):
        for word in dict:
            for i in range(1, self.ngram_lim+1):
                for j in range(0, len(word)-i+1):
                    if(word[j:j+i] not in self.ngram):
                        self.ngram[word[j:j+i]] = 0
                    self.ngram[word[j:j+i]] += 1


    def compute_ngrams(self, word):
        for n in range (1, self.ngram_lim+1):
            letters = {}
            for c in range(ord('a'),ord('z')+1):
                if(chr(c) not in self.guessed_letters):
                    letters[chr(c)] = 0

            letters_total = 0
            for missing in range (0, n):
                for i in range (0, len(word)-n+1):
                    good = True
                    for j in range (0, n):
                        if(j == missing):
                            good = good and (word[i+j] == '.')
                        else:
                            good = good and (word[i+j] != '.')
                    if(good == True):
                        for c in range(ord('a'),ord('z')+1):
                            if(chr(c) not in self.guessed_letters):
                                substr = ""
                                for k in range (0, n):
                                    if(k == missing):
                                        substr += chr(c)
                                    else:
                                        substr += word[i+k]
                                if(substr in self.ngram):
                                    letters[chr(c)] += self.ngram[substr]
                                    letters_total += self.ngram[substr]

            for key, value in letters.items():
                if(letters_total > 0):
                    self.prob[key] += self.weights[n] * letters[key] / letters_total
        
        best_letter = ''
        best_prob = 0
        for key, value in self.prob.items():
            if(value > best_prob):
                best_prob = value
                best_letter = key

        return best_letter
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

## Results:
With the training done on a 250,000 word dataset, this algorithm was able to predict words from a mutually exclusive dataset with an overall success rate of 63%, given a total of six tries to guess the word.
